In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!pip install tensorflow_addons

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv1D, Input, Dense, Add, Multiply, concatenate, Reshape
from tensorflow.keras.layers import Embedding, Bidirectional
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import losses, models, optimizers
import tensorflow_addons as tfa

In [0]:
# =====================================================================================
# Directory Settings
# =====================================================================================
DRIVE_PATH = './drive/My Drive/Colab Notebooks/Ion/'
ROOT = DRIVE_PATH+'input/liverpool-ion-switching/'
CLEAN_ROOT = DRIVE_PATH+'input/data-without-drift/'
OUTPUT_DIR = DRIVE_PATH+'keras1/'

In [0]:
class CFG:
    learning_rate=1e-3
    batch_size=16
    num_workers=4
    num_train_epochs=2 #150
    weight_decay=0.01
    dropout=0.3
    emb_size=100
    hidden_size=164
    seq_len=5000
    total_cate_size=40
    seed=777
    encoder='Wavenet_keras'
    target_size=11
    n_fold=4
    fold=[0, 1, 2, 3]

In [0]:
# =====================================================================================
# Library
# =====================================================================================
import sys
import os
import gc
import time
from contextlib import contextmanager
from collections import Counter, defaultdict

import numpy as np
import pandas as pd
import random
import json

from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn import preprocessing

from tqdm import tqdm, tqdm_notebook

import torch

import warnings
warnings.filterwarnings("ignore")

# =====================================================================================
# Utils
# =====================================================================================
def get_logger(filename=OUTPUT_DIR+'log'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = get_logger()


@contextmanager
def timer(name):
    t0 = time.time()
    logger.info(f'[{name}] start')
    yield
    logger.info(f'[{name}] done in {time.time() - t0:.0f} s')


# for tf
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)


def load_df(path, debug=False):
    # load df .csv or .pkl
    if path.split('.')[-1]=='csv':
        df = pd.read_csv(path)
        if debug:
            df = pd.read_csv(path, nrows=1000)
    elif path.split('.')[-1]=='pkl':
        df = pd.read_pickle(path)
    print(f"{path} shape / {df.shape} ")
    return df


def make_stratified_group_k_folds(_df, _id, target, group, k, seed=42, save_path=OUTPUT_DIR+'folds.csv'):

    def stratified_group_k_fold(X, y, groups, k, seed=42):

        """
        original author : jakubwasikowski
        reference : https://www.kaggle.com/jakubwasikowski/stratified-group-k-fold-cross-validation
        """

        labels_num = np.max(y) + 1
        y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
        y_distr = Counter()
        for label, g in zip(y, groups):
            y_counts_per_group[g][label] += 1
            y_distr[label] += 1

        y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
        groups_per_fold = defaultdict(set)

        def eval_y_counts_per_fold(y_counts, fold):
            y_counts_per_fold[fold] += y_counts
            std_per_label = []
            for label in range(labels_num):
                label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
                std_per_label.append(label_std)
            y_counts_per_fold[fold] -= y_counts
            return np.mean(std_per_label)

        groups_and_y_counts = list(y_counts_per_group.items())
        random.Random(seed).shuffle(groups_and_y_counts)

        for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
            best_fold = None
            min_eval = None
            for i in range(k):
                fold_eval = eval_y_counts_per_fold(y_counts, i)
                if min_eval is None or fold_eval < min_eval:
                    min_eval = fold_eval
                    best_fold = i
            y_counts_per_fold[best_fold] += y_counts
            groups_per_fold[best_fold].add(g)

        all_groups = set(groups)
        for i in range(k):
            train_groups = all_groups - groups_per_fold[i]
            test_groups = groups_per_fold[i]

            train_indices = [i for i, g in enumerate(groups) if g in train_groups]
            test_indices = [i for i, g in enumerate(groups) if g in test_groups]

            yield train_indices, test_indices

    df = _df.copy()
    le = preprocessing.LabelEncoder()
    groups = le.fit_transform(df[group].values)
    for n, (train_index, val_index) in enumerate(stratified_group_k_fold(df, df[target], groups, k=k, seed=seed)):
        df.loc[val_index, 'fold'] = int(n)
    df['fold'] = df['fold'].astype(int)
    df[[_id, target, group, 'fold']].to_csv(save_path, index=None)

    return df[[_id, target, group, 'fold']]

In [7]:
# =====================================================================================
# General Settings
# =====================================================================================
df_path_dict = {'train': CLEAN_ROOT+'train_clean.csv',
                'test': CLEAN_ROOT+'test_clean.csv',
                'sample_submission': ROOT+'sample_submission.csv'}
ID = 'time'
TARGET = 'open_channels'
seed_everything(seed=CFG.seed)


# =====================================================================================
# Data Loading
# =====================================================================================
with timer('Data Loading'):
    X_train = load_df(path=df_path_dict['train'])
    #X_test = load_df(path=df_path_dict['test'])
    #submission = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})
    oof_lgb = pd.read_csv(DRIVE_PATH+'input/ion-oof/oof_lightgbm.csv').rename(columns={TARGET:'signal_lgb_oof'})
    oof_cat = pd.read_csv(DRIVE_PATH+'input/ion-oof/oof_catboost.csv').rename(columns={TARGET:'signal_cat_oof'})
    oof_xgb = pd.read_csv(DRIVE_PATH+'input/ion-oof/oof_xgb.csv').rename(columns={TARGET:'signal_xgb_oof'})

    X_train = pd.concat([X_train, oof_lgb['signal_lgb_oof']], axis=1)
    X_train = pd.concat([X_train, oof_cat['signal_cat_oof']], axis=1)
    X_train = pd.concat([X_train, oof_xgb['signal_xgb_oof']], axis=1)

    del oof_lgb, oof_cat, oof_xgb; gc.collect()

[Data Loading] start


./drive/My Drive/Colab Notebooks/Ion/input/data-without-drift/train_clean.csv shape / (5000000, 3) 


[Data Loading] done in 6 s


In [0]:
# =====================================================================================
# Preprocess
# =====================================================================================
X_train['batch'] = X_train.index // 500000
X_train['batch'] = X_train['batch'].astype(int)


def signal_2_features(X_train, X_test=None):
    X_train['signal_2'] = X_train['signal'] ** 2
    mean_signal_2 = X_train['signal_2'].values.mean()
    X_train['signal_2_mean_diff'] = X_train['signal_2'] - mean_signal_2
    if X_test is not None:
        X_test['signal_2'] = X_test['signal'] ** 2
        X_test['signal_2_mean_diff'] = X_test['signal_2'] - mean_signal_2
        return  X_train, X_test
    return X_train

#X_train = signal_2_features(X_train, X_test=None)


def signal2cate(X_train, X_test=None, NUM_BINS=1000):
    signal_bins = np.linspace(X_train['signal'].min(), X_train['signal'].max(), NUM_BINS + 1)
    train_signal_dig = np.digitize(X_train['signal'], bins=signal_bins) - 1
    train_signal_dig = np.minimum(train_signal_dig, len(signal_bins) - 2)
    X_train['signal_cate'] = train_signal_dig
    if X_test is not None:
        test_signal_dig = np.digitize(X_test['signal'], bins=signal_bins) - 1
        test_signal_dig = np.minimum(test_signal_dig, len(signal_bins) - 2)
        X_test['signal_cate'] = test_signal_dig
        return  X_train, X_test
    return X_train

X_train = signal2cate(X_train, X_test=None, NUM_BINS=CFG.total_cate_size)


def add_num_features(X_train, X_test=None):
    max_signal = X_train['signal'].max()
    min_signal = X_train['signal'].min()
    X_train['signal_diff_max'] = max_signal - X_train['signal']
    X_train['signal_diff_min'] = min_signal - X_train['signal']
    if X_test is not None:
        X_test['signal_diff_max'] = max_signal - X_test['signal']
        X_test['signal_diff_min'] = min_signal - X_test['signal']
        return  X_train, X_test
    return X_train

X_train = add_num_features(X_train, X_test=None)


def calc_gradients(df):

    df['signal_gradient'] = np.gradient(df['signal'].values)

    return df

def preprocess_df(df):

    output = pd.DataFrame()

    for i in range(int(len(df)/500000)):
        tmp = df.loc[i * 500000: 500000*(i + 1) - 1].reset_index(drop=True)
        tmp = calc_gradients(tmp)
        output = pd.concat([output, tmp])

    return output.reset_index(drop=True)

X_train = preprocess_df(X_train)

In [0]:
# =====================================================================================
# Train functions
# =====================================================================================
import math
from sklearn.metrics import f1_score

def lr_schedule(epoch):
    if epoch < 30:
        lr = CFG.learning_rate
    elif epoch < 40:
        lr = CFG.learning_rate / 3
    elif epoch < 50:
        lr = CFG.learning_rate / 5
    elif epoch < 60:
        lr = CFG.learning_rate / 7
    elif epoch < 70:
        lr = CFG.learning_rate / 9
    elif epoch < 80:
        lr = CFG.learning_rate / 11
    elif epoch < 90:
        lr = CFG.learning_rate / 13
    else:
        lr = CFG.learning_rate / 100
    return lr

class MacroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = np.argmax(targets, axis=2).reshape(-1)
        
    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        score = f1_score(self.targets, pred, average='macro')
        print(f' - F1Score: {score:.5f}')

In [0]:
# =====================================================================================
# Wavenet Model
# =====================================================================================
"""
import torch.nn as nn
import torch.nn.functional as F


class wave_block(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation):
        super().__init__()
        self.kernel_size = kernel_size
        self.in_ch = in_ch
        self.out_ch = out_ch
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size, padding=int((kernel_size + (kernel_size-1)*(dilation-1))/2), dilation=dilation)
        self.conv3 = nn.Conv1d(out_ch, out_ch, kernel_size, padding=int((kernel_size + (kernel_size-1)*(dilation-1))/2), dilation=dilation)
        self.conv4 = nn.Conv1d(out_ch, out_ch, 1)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.bn = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        res_x = x
        tanh = self.tanh(self.conv2(x))
        sig = self.sigmoid(self.conv3(x))
        res = tanh.mul(sig)
        x = self.conv4(res)
        x = res_x + x
        return x


class Wavenet(nn.Module):
    def __init__(self, cfg, basic_block=wave_block):
        super().__init__()
        self.cfg = cfg
        self.basic_block = basic_block
        cont_col_size = len(cfg.cont_cols)
        cate_col_size = len(cfg.cate_cols)
        self.cate_emb = nn.Embedding(cfg.total_cate_size, cfg.emb_size, padding_idx=0)
        #self.cate_proj = nn.Sequential(
        #    nn.Linear(cfg.emb_size*cate_col_size, cfg.hidden_size//2),
        #    nn.LayerNorm(cfg.hidden_size//2),
        #)
        self.layer1 = self._make_layers(cont_col_size, cfg.hidden_size//16, 3, 12)
        self.layer2 = self._make_layers(cfg.hidden_size//16, cfg.hidden_size//8, 3, 8)
        self.layer3 = self._make_layers(cfg.hidden_size//8, cfg.hidden_size//4, 3, 4)
        self.layer4 = self._make_layers(cfg.hidden_size//4, cfg.hidden_size//2, 3, 1)
        self.gru = nn.GRU(input_size=cfg.emb_size*cate_col_size+cont_col_size, hidden_size=cfg.hidden_size//4, num_layers=cfg.nlayers,
                          bidirectional=True, batch_first=True, dropout=cfg.dropout)
        def get_reg():
            return nn.Sequential(
            nn.Linear(cfg.hidden_size, cfg.hidden_size),
            nn.LayerNorm(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.ReLU(),
            nn.Linear(cfg.hidden_size, cfg.hidden_size),
            nn.LayerNorm(cfg.hidden_size),
            nn.Dropout(cfg.dropout),
            nn.ReLU(),
            nn.Linear(cfg.hidden_size, cfg.target_size),
        )
        self.fc = get_reg()

    def _make_layers(self, in_ch, out_ch, kernel_size, n):
        dilation_rates = [2 ** i for i in range(n)]
        layers = [nn.Conv1d(in_ch, out_ch, 1)]
        for dilation in dilation_rates:
            layers.append(self.basic_block(out_ch, out_ch, kernel_size, dilation))
        return nn.Sequential(*layers)

    def forward(self, cate_x, cont_x):
        batch_size = cate_x.size(0)
        # RNN
        cate_emb = self.cate_emb(cate_x).view(batch_size, self.cfg.seq_len, -1)
        #h_gru, _ = self.gru(cate_emb)
        seq_emb = torch.cat((cate_emb, cont_x), 2)
        h_gru, _ = self.gru(seq_emb)
        # CNN
        cont_x = cont_x.permute(0, 2, 1)
        cont_x = self.layer1(cont_x)
        cont_x = self.layer2(cont_x)
        cont_x = self.layer3(cont_x)
        cont_x = self.layer4(cont_x)
        # CNN & RNN
        x = torch.cat((cont_x, h_gru.permute(0, 2, 1)), 1).permute(0, 2, 1)
        # fc
        x = self.fc(x)
        return x
"""


def Wavenet_keras(cate_cols, cont_cols):
    
    def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = Multiply()([tanh_out, sigm_out])
            x = Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = Add()([res_x, x])
        return res_x
    
    input1 = Input(shape=(None, len(cont_cols)))
    input2 = Input(shape=(None, len(cate_cols)))
    # RNN
    cate_emb = Embedding(CFG.total_cate_size, CFG.emb_size)(input2)
    cate_emb = Reshape((-1, CFG.emb_size))(cate_emb)
    seq_emb = concatenate([cate_emb, input1])
    h_gru = Bidirectional(CuDNNGRU(CFG.hidden_size//4, return_sequences=True))(seq_emb) # dropout?
    # CNN
    cont_x = wave_block(input1, CFG.hidden_size//16, 3, 12)
    cont_x = wave_block(cont_x, CFG.hidden_size//8, 3, 8)
    cont_x = wave_block(cont_x, CFG.hidden_size//4, 3, 4)
    cont_x = wave_block(cont_x, CFG.hidden_size//2, 3, 1)
    # CNN & RNN
    x = concatenate([cont_x, h_gru])
    # fc
    x = Dense(11, activation='softmax', name='out')(x)
    
    model = models.Model(inputs=[input1, input2], outputs=x)
    
    opt = Adam(lr=CFG.learning_rate)
    opt = tfa.optimizers.SWA(opt)
    model.compile(loss=losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

    return model

In [0]:
# =====================================================================================
# Get Sample function
# =====================================================================================
def tta_group(X_train):
    X_train = X_train.reset_index()
    tmp = pd.DataFrame()
    for i in tqdm(range(int(len(X_train)/500000))):
        _tmp = X_train[i*500000+CFG.seq_len//2:(i+1)*500000-CFG.seq_len//2].reset_index(drop=True)
        tmp = pd.concat([tmp, _tmp])
    tmp = tmp.reset_index(drop=True)
    tmp['tta_group'] = tmp.index // CFG.seq_len
    tmp['tta_group'] = tmp['tta_group'] + X_train['group'].nunique()
    X_train = X_train.merge(tmp[['index', 'tta_group']], on='index', how='left').fillna(-1).drop(columns='index')
    return X_train


def get_sample_indices(df):
    sample_indices = []
    group_indices = []
    df_groups = df.groupby('group').groups
    tta_df_groups = df[df['tta_group']>=0].groupby('tta_group').groups
    for group_idx, indices in enumerate(df_groups.values()):
        sample_indices.append(indices.values)
        group_indices.append(group_idx)
    for group_idx, indices in enumerate(tta_df_groups.values()):
        sample_indices.append(indices.values)
        group_indices.append(group_idx+len(df_groups))
    return np.array(sample_indices), group_indices

In [12]:
# =====================================================================================
# Train loop
# =====================================================================================
def main(X_train):

    seed_everything(CFG.seed)
    K.clear_session()
    config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
    tf.compat.v1.keras.backend.set_session(sess)

    # =====================================================================================
    # Settings
    # =====================================================================================
    cate_cols = [c for c in X_train.columns if c.find('cate')>=0]
    cont_cols = [c for c in X_train.columns if c.find('signal')>=0]
    cont_cols = [c for c in cont_cols if c not in cate_cols]
    logger.info(f'cont_cols: {cont_cols}')
    logger.info(f'cate_cols: {cate_cols}')
    target_df = pd.get_dummies(X_train['open_channels'].values)
    cont_df = X_train[cont_cols]
    cate_df = X_train[cate_cols]

    # =====================================================================================
    # run function
    # =====================================================================================
    def run(fold, trn_idx, val_idx):

        train_cont_x = np.zeros((len(trn_idx), CFG.seq_len, len(cont_cols)))
        train_cate_x = np.zeros((len(trn_idx), CFG.seq_len, len(cate_cols)))
        train_y = np.zeros((len(trn_idx), CFG.seq_len, 11))
        for i in tqdm_notebook(range(len(trn_idx))):
            train_cont_x[i] = cont_df.iloc[sample_indices[trn_idx[i]]].values
            train_cate_x[i] = cate_df.iloc[sample_indices[trn_idx[i]]].values     
            train_y[i] = target_df.iloc[sample_indices[trn_idx[i]]].values
        
        val_cont_x = np.zeros((len(val_idx), CFG.seq_len, len(cont_cols)))
        val_cate_x = np.zeros((len(val_idx), CFG.seq_len, len(cate_cols)))
        val_y = np.zeros((len(val_idx), CFG.seq_len, 11))
        for i in tqdm_notebook(range(len(val_idx))):
            val_cont_x[i] = cont_df.iloc[sample_indices[val_idx[i]]].values
            val_cate_x[i] = cate_df.iloc[sample_indices[val_idx[i]]].values     
            val_y[i] = target_df.iloc[sample_indices[val_idx[i]]].values

        gc.collect()

        # =====================================================================================
        # Training loop
        # =====================================================================================
        model = Wavenet_keras(cate_cols, cont_cols)
        cb_lr_schedule = LearningRateScheduler(lr_schedule)
        loss_saving_callback = keras.callbacks.ModelCheckpoint(OUTPUT_DIR+f'keras_fold{fold}_best_loss.h5', monitor='val_loss', verbose=1, 
                                                               save_best_only=True, save_weights_only=True)
        score_saving_callback = keras.callbacks.ModelCheckpoint(OUTPUT_DIR+f'keras_fold{fold}_best_score.h5', monitor='val_accuracy', verbose=1, 
                                                                save_best_only=True, save_weights_only=True)
        # ModelCheckpoint追加するとMacroF1Callbackが出力されない...
        model.fit([train_cont_x, train_cate_x], train_y, epochs=CFG.num_train_epochs,
                  #callbacks=[cb_lr_schedule, loss_saving_callback, score_saving_callback, MacroF1(model, [val_cont_x, val_cate_x], val_y)],
                  callbacks=[cb_lr_schedule, loss_saving_callback, score_saving_callback],
                  batch_size=CFG.batch_size, verbose=1, validation_data=([val_cont_x, val_cate_x], val_y))
        #model.save_weights(OUTPUT_DIR+f'keras_fold{fold}.h5')

        # =====================================================================================
        # Evaluation
        # =====================================================================================
        model = Wavenet_keras(cate_cols, cont_cols)
        model.load_weights(OUTPUT_DIR+f'keras_fold{fold}_best_score.h5')
        preds_f = model.predict([val_cont_x, val_cate_x])
        predictions = np.argmax(preds_f, axis=2).reshape(-1)
        groundtruth = np.argmax(val_y, axis=2).reshape(-1)
        score = f1_score(predictions, groundtruth, labels=list(range(11)), average='macro')
        logger.info(f'Training fold {fold} completed. macro f1 score : {score :1.5f}')

        return predictions, groundtruth

    # =====================================================================================
    # k-fold
    # =====================================================================================
    X_train['group'] = X_train.index // CFG.seq_len
    X_train = tta_group(X_train)
    X_train['tta_group'] = X_train['tta_group'].astype(int)
    X_train['split_group'] = X_train.index // (CFG.seq_len*2)
    sample_indices, group_indices = get_sample_indices(X_train)
    print(len(group_indices))
    print(len(set(group_indices)))
    group_map = dict(X_train[['group', 'split_group']].values.tolist())
    tta_group_map = dict(X_train[['tta_group', 'split_group']].values.tolist())
    group_map.update(tta_group_map)
    group_indices = [group_map[i] for i in group_indices]
    print(len(group_indices))
    print(len(set(group_indices)))
    folds = make_stratified_group_k_folds(X_train, _id='time', target='open_channels', group='split_group', k=CFG.n_fold, seed=CFG.seed)
    #folds = pd.read_csv('../input/ion-folds/folds.csv')
    #folds = pd.read_csv(DRIVE_PATH+'input/ion-folds/folds.csv')
    folds_map = dict(folds[['split_group', 'fold']].values.tolist())
    group_indices = [folds_map[i] for i in group_indices]
    skf = GroupKFold(n_splits=CFG.n_fold)
    splits = [x for x in skf.split(sample_indices, None, group_indices)]
    predictions, groundtruth = [], []
    for fold, (trn_idx, val_idx) in enumerate(splits):
        if fold in CFG.fold:
            # without leak tta for val_idx
            normal_val_idx = val_idx[val_idx<(5000000//CFG.seq_len)]
            tta_val_idx = val_idx[val_idx>=(5000000//CFG.seq_len)]
            no_leak_tta_val_idx = [i for i in tta_val_idx if (i-1000)%99%2==0]
            val_idx = list(tta_val_idx) + list(no_leak_tta_val_idx)
            print(f'len(val_idx): {len(val_idx)}')
            with timer(f'##### Running Fold: {fold} #####'):
                _predictions, _groundtruth = run(fold, trn_idx, val_idx)
                predictions.append(_predictions)
                groundtruth.append(_groundtruth)
    predictions = np.concatenate(predictions)
    groundtruth = np.concatenate(groundtruth)
    score = f1_score(predictions, groundtruth, labels=list(range(11)), average='macro')
    logger.info(f'##### CV Score: {score} #####')


if __name__ == '__main__':
    main(X_train)

cont_cols: ['signal', 'signal_lgb_oof', 'signal_cat_oof', 'signal_xgb_oof', 'signal_diff_max', 'signal_diff_min', 'signal_gradient']
cate_cols: ['signal_cate']
100%|██████████| 10/10 [00:02<00:00,  3.74it/s]


1990
1990
1990
500


[##### Running Fold: 0 #####] start


len(val_idx): 374



Epoch 1/2
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94/94 [==============================] - ETA: 0s - loss: 0.6172 - accuracy: 0.8099
Epoch 00001: val_loss improved from inf to 0.16528, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold0_best_loss.h5

Epoch 00001: val_accuracy improved from -inf to 0.95358, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold0_best_score.h5
 - F1Score: 0.83902
94/94 [==============================] - 61s 645ms/step - loss: 0.6172 - accuracy: 0.8099 - val_loss: 0.1653 - val_accuracy: 0.9536 - lr: 0.0010
Epoch 2/2
94/94 [==============================] - ETA: 0s - loss: 0.1169 - accuracy: 0.9629
Epoch 00002: val_loss improved from 0.16528 to 0.10354, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold0_best_loss.h5

Epoch 00002: val_accuracy improved from 0.95358 to 0.96646, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold0_best_score.h

...


Epoch 1/2
94/94 [==============================] - ETA: 0s - loss: 0.6009 - accuracy: 0.8174
Epoch 00001: val_loss improved from inf to 0.16300, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold2_best_loss.h5

Epoch 00001: val_accuracy improved from -inf to 0.95726, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold2_best_score.h5
 - F1Score: 0.88690
94/94 [==============================] - 49s 517ms/step - loss: 0.6009 - accuracy: 0.8174 - val_loss: 0.1630 - val_accuracy: 0.9573 - lr: 0.0010
Epoch 2/2
94/94 [==============================] - ETA: 0s - loss: 0.1149 - accuracy: 0.9636
Epoch 00002: val_loss improved from 0.16300 to 0.10398, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold2_best_loss.h5

Epoch 00002: val_accuracy improved from 0.95726 to 0.96634, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold2_best_score.h5
 - F1Score: 0.93349
94/94 [==============================] - 40s 427ms/step - l

Training fold 2 completed. macro f1 score : 0.93349
[##### Running Fold: 2 #####] done in 132 s
[##### Running Fold: 3 #####] start


len(val_idx): 372



Epoch 1/2
94/94 [==============================] - ETA: 0s - loss: 0.4643 - accuracy: 0.8582
Epoch 00001: val_loss improved from inf to 0.12133, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold3_best_loss.h5

Epoch 00001: val_accuracy improved from -inf to 0.96471, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold3_best_score.h5
 - F1Score: 0.92289
94/94 [==============================] - 48s 516ms/step - loss: 0.4643 - accuracy: 0.8582 - val_loss: 0.1213 - val_accuracy: 0.9647 - lr: 0.0010
Epoch 2/2
94/94 [==============================] - ETA: 0s - loss: 0.1016 - accuracy: 0.9658
Epoch 00002: val_loss improved from 0.12133 to 0.09940, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold3_best_loss.h5

Epoch 00002: val_accuracy improved from 0.96471 to 0.96702, saving model to ./drive/My Drive/Colab Notebooks/Ion/keras1/keras_fold3_best_score.h5
 - F1Score: 0.93389
94/94 [==============================] - 40s 427ms/step - l

Training fold 3 completed. macro f1 score : 0.93389
[##### Running Fold: 3 #####] done in 132 s
##### CV Score: 0.9335305480196591 #####
